In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from collections import defaultdict

from model_io.Base import Base
from model_io.Devices import Devices
from model_io.Activities import Activities

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

from IPython.display import display

DB='postgresql+psycopg2:///ucnstudy_hostview_data'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine)

In [2]:
ses = Session()
devices = ses.query(Devices)

for device in devices:
    #select only users from ucnstudy
    if device.id == 5 or device.id == 6 or device.id == 8 or device.id == 11 or device.id == 12:
        print (device.device_id + '===============')

        sql_beg_day = text('SELECT distinct session_id, activities.logged_at \
        FROM activities join \
        (SELECT DATE(logged_at) as date_entered, MIN(logged_at) as min_time \
        FROM activities \
        WHERE session_id =:dev_id  and fullscreen = 1 and extract (hour from logged_at) > 3 \
        GROUP BY date(logged_at)) AS grp ON grp.min_time = activities.logged_at \
        order by activities.logged_at;').bindparams(dev_id = device.id)
        result_beg_day = ses.execute(sql_beg_day)
            
        sql_end_day = text('SELECT distinct session_id, activities.finished_at \
        FROM activities join \
        (SELECT DATE(finished_at) as date_entered, MAX(finished_at) as max_time \
        FROM activities \
        WHERE session_id =:dev_id and fullscreen = 1 \
        GROUP BY date(finished_at)) AS grp ON grp.max_time = activities.finished_at \
        order by activities.finished_at;').bindparams(dev_id = device.id)
        result_end_day = ses.execute(sql_end_day)

        sql_beg_day_nolimit = text('SELECT distinct session_id, activities.logged_at \
        FROM activities join \
        (SELECT DATE(logged_at) as date_entered, MIN(logged_at) as min_time \
        FROM activities \
        WHERE session_id =:dev_id  and fullscreen = 1 \
        GROUP BY date(logged_at)) AS grp ON grp.min_time = activities.logged_at \
        order by activities.logged_at;').bindparams(dev_id =device.id)
        result_beg_day_nolimit = ses.execute(sql_beg_day_nolimit)

        #organize data
        info_end = defaultdict(list)
        for row in result_end_day:
            info_end['devid'].append(row[0])
            info_end['ts_end'].append(row[1])

        info_beg = defaultdict(list)
        for row in result_beg_day:
            info_beg['devid'].append(row[0])
            info_beg['ts_start'].append(row[1])

        #add days that only have value before 3 am
        for row in result_beg_day_nolimit:
            timst = row[1]
            in_list = False
            for dt in info_beg['ts_start']:
                if dt.date() == timst.date():
                    in_list = True
            if in_list == False:
                #insert in the correct position
                cont = 0
                for dat in info_beg['ts_start']:
                    if timst.date() > dat.date():
                        cont = cont + 1
                        info_beg['ts_start'].insert(cont, timst)
                        info_beg['devid'].insert(cont, row[0])


        df_beg = pd.DataFrame(info_beg)
        display(df_beg)
        df_end = pd.DataFrame(info_end)
        display(df_end)
        
        
        #creates table per day for beginning
        info_week = defaultdict(list)
        if (info_beg['ts_start']):
            for timst in info_beg['ts_start']:
                day = timst
                weekday = day.strftime('%A')
                info_week[weekday].append(day)
         
        print (device.device_id + '===============')
        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col[name+' start'] = info_week[name]
            df_week = pd.DataFrame(df_col)
            display(df_week)
            
            
        #creates table per day for ending
        info_week = defaultdict(list)
        if (info_end['ts_end']):
            for timst in info_end['ts_end']:
                day = timst
                weekday = day.strftime('%A')
                info_week[weekday].append(day)
             
        print (device.device_id + '===============')
        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col[name+' end'] = info_week[name] 
            df_week = pd.DataFrame(df_col)
            display(df_week)
        

kemianny.mainlaptop===============


,devid,ts_start
0,5,2015-11-20 16:55:21.403
1,5,2015-11-23 16:16:23.507
2,5,2015-11-25 16:22:08.327


,devid,ts_end
0,5,2015-11-20 23:42:14.830
1,5,2015-11-23 19:23:55.402
2,5,2015-11-25 16:22:08.527


kemianny.mainlaptop===============


,Monday start
0,2015-11-23 16:16:23.507


...

,Friday end
0,2015-11-13 09:33:59.106
1,2015-11-20 22:53:42.069


,Wednesday end
0,2015-11-18 21:37:14.355
1,2015-11-25 21:38:57.521


,Thursday end
0,2015-11-12 22:03:58.187
1,2015-11-19 23:59:43.727
2,2015-11-26 22:06:32.730


,Sunday end
0,2015-11-15 21:16:12.166
1,2015-11-22 23:59:40.547


,Saturday end
0,2015-11-14 11:53:26.592
1,2015-11-21 20:43:24.515
